# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [60]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
from torchvision import datasets, transforms, models
from PIL import Image
from workspace_utils import active_session

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [61]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [62]:
# TODO: Define your transforms for the training, validation, and testing sets
data_transforms = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

test_data_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

# TODO: Load the datasets with ImageFolder
image_datasets = {}
image_datasets['train'] = datasets.ImageFolder(train_dir, transform=data_transforms)
image_datasets['test'] = datasets.ImageFolder(test_dir, transform=test_data_transforms)
image_datasets['validate'] = datasets.ImageFolder(test_dir, transform=test_data_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True)
dataloaders['test'] = torch.utils.data.DataLoader(image_datasets['test'], batch_size=64, shuffle=True)
dataloaders['validate'] = torch.utils.data.DataLoader(image_datasets['validate'], batch_size=64, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # select cuda as default device

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [63]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [64]:
# TODO: Build and train your network
def get_model():
    model = models.vgg19(pretrained=True)

    # Turns off gradient for existing trained model
    for p in model.parameters():
        p.requires_grad = False

    classifier = nn.Sequential(
        nn.Linear(25088, 1000),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(1000, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, 102),
        nn.LogSoftmax(dim=1)
    )

    model.classifier = classifier
    
    return model

In [11]:
model = get_model()  # Create model
criterion = nn.NLLLoss()  # Loss funtion
optimiser = optim.Adam(model.classifier.parameters(), lr=0.002)  # Optimiser for enabling gradient
model.to(device);  # Set default model device

In [12]:
# Train network
with active_session():
    epochs = 30
    print_every = 5
    

    for i in range(epochs):
        running_loss = 0
        step = 0
        
        for images, labels in dataloaders['train']:        
            images, labels = images.to(device), labels.to(device)
            optimiser.zero_grad()  # Reset gradient to zero

            output = model(images)
            loss = criterion(output, labels)

            loss.backward()
            optimiser.step()

            running_loss += loss.item()
            step += 1

            if step % print_every == 0:
                running_loss = running_loss/print_every
                print('Loss: {}'.format(running_loss))

Loss: 5.275033187866211
Loss: 5.454801979064941
Loss: 5.446021545410156
Loss: 5.322399900054931
Loss: 5.098110446472168
Loss: 4.7861542719879155
Loss: 4.366793680554199
Loss: 4.354018602443848
Loss: 4.216671783461792
Loss: 3.788381258395239
Loss: 3.690286811142549
Loss: 3.5693918100892277
Loss: 3.5464216874420402
Loss: 3.454838689295049
Loss: 3.3590877406055917
Loss: 3.3051223348032472
Loss: 3.172865144222002
Loss: 3.0300472720084626
Loss: 2.850518102320394
Loss: 2.948568557674162
Loss: 2.4074642658233643
Loss: 2.820886254310608
Loss: 2.815117859840393
Loss: 2.797074751853943
Loss: 2.742703715324402
Loss: 2.8399741250991823
Loss: 2.893853704109192
Loss: 2.642031129196167
Loss: 2.7114904276519773
Loss: 2.7072915623980713
Loss: 2.517600265947937
Loss: 2.591381257336926
Loss: 2.658764470728555
Loss: 2.7266998269383746
Loss: 2.577344679399821
Loss: 2.5057744549839684
Loss: 2.407241264325956
Loss: 2.3887686092342095
Loss: 2.495555554999613
Loss: 2.626143677070479
Loss: 1.895795726776123
Los

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [69]:
# TODO: Do validation on the test set
model.eval()
accuracy = 0
test_loss = 0

with torch.no_grad():
    
    for images, labels in dataloaders['test']:
        images, labels = images.to(device), labels.to(device)
        output = model(images)
        test_loss += criterion(output, labels).item()
        
        output = torch.exp(output)
        top_prb, top_class = output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
        
test_loss = test_loss/len(dataloaders['test'])
accuracy = accuracy/len(dataloaders['test'])

print('Accuracy: {}%'.format(accuracy * 100))
print('Test Loss: {}'.format(test_loss))
print('----------------------------------------------')

model.train();

Accuracy: 81.06146317261917%
Test Loss: 0.755593845477471
----------------------------------------------


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [14]:
# TODO: Save the checkpoint
checkpoint = {
    'input_size': 25088,
    'output_size': 102,
    'class_to_idx': image_datasets['train'].class_to_idx,
    'state_dict': model.state_dict(),
    'optimiser_dict': optimiser.state_dict(),
    'epochs': 10
}

# torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [68]:
# Load checkpoint
def load_from_checkpoint(checkpoint='checkpoint.pth'):
    model = get_model()
    optimiser = optim.Adam(model.classifier.parameters(), lr=0.0)

    checkpoint = torch.load(checkpoint)
    model.load_state_dict(checkpoint['state_dict'])
    model.class_to_idx = checkpoint['class_to_idx']
    optimiser.load_state_dict(checkpoint['optimiser_dict'])
    
    return model, optimiser

model, optimiser = load_from_checkpoint(checkpoint='checkpoint.pth')
criterion = nn.NLLLoss()
model.to(device);

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [70]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    pil_image = Image.open(image)
    pil_image.thumbnail((255, 255))
    width, height = pil_image.size
    
    top_crop = (height - 224) / 2
    right_crop = (width + 224) / 2
    bottom_crop = (height + 224) / 2
    left_crop = (width - 224) / 2
    crop_size = (left_crop, top_crop, right_crop, bottom_crop)
    pil_image = pil_image.crop(crop_size)
    
    np_image = np.array(pil_image) / 255
    np_image = np_image - np.array([0.485, 0.456, 0.406])
    np_image = np_image / np.array([0.229, 0.224, 0.225])
    np_image = np.transpose(np_image, (2, 0, 1))
    
    t_image = torch.from_numpy(np_image)
    return t_image.float()

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [71]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [74]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file
    image = process_image(image_path)
    image.unsqueeze_(0)
    
    classifier, optimiser = load_from_checkpoint('checkpoint.pth')
    classifier.to(device)
    image = image.to(device)
    
    labels = [None] * len(classifier.class_to_idx)
    
    for key in classifier.class_to_idx:
        labels[classifier.class_to_idx[key]] = key
        
    classifier.eval()
    with torch.no_grad():
        output = torch.exp(classifier(image))
        top_p, top_class = output.topk(topk, dim=1)
        
    classifier.train()
    top_p = top_p.cpu().numpy().flatten()
    top_class = [labels[i] for i in top_class.cpu().numpy().flatten()]
    
    return top_p, top_class

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes
image_path = 'flowers/test/100/image_07926.jpg'
image = process_image(image_path)
probs, classes = predict(image_path, 'checkpoint.pth')

fig = plt.figure(figsize = [5, 10])

ax = plt.subplot(2, 1, 1)
ax.tick_params(axis='both', length=0)
ax.set_xticklabels('')
ax.set_yticklabels('')
imshow(image, ax)

ax2 = plt.subplot(2, 1, 2)
labels = [cat_to_name[i] for i in classes]
base_color = sb.color_palette()[0]
data = pd.DataFrame({
    'Class': labels,
    'Probability': probs,
})
sb.barplot(data = data, x = 'Probability', y = 'Class', color = base_color)